# CNN decoder

Author: YinTaiChen

In [1]:
import torch
import torchvision
import torchvision.transforms as transforms

In [2]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5))]
)

trainset = torchvision.datasets.CIFAR10(root="./data", train=True, download=True, transform=transform)

trainloader = torch.utils.data.DataLoader(trainset, batch_size=16, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root="./data", train=False, download=True, transform=transform)

testloader = torch.utils.data.DataLoader(testset, batch_size=16, shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [3]:
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F

class cnnDecoder(nn.Module):
    def __init__(self):
        super(cnnDecoder, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        self.decoder = nn.Sequential(
            nn.Linear(84, 120), # fc2
            nn.ReLU(),
            nn.Linear(120, 16 * 5 * 5), # fc1
            nn.Linear(16 * 5 * 5, 16 * 10 * 10), # pool
            nn.ReLU(),
            nn.Linear(16 * 10 * 10, 6 * 14 * 14), # conv2
            nn.Linear(6 * 14 * 14, 6 * 28 * 28), # pool
            nn.ReLU(),
            nn.Linear(6 * 28 * 28, 3 * 32 * 32), # conv1
        )
        
    def forward(self, x, phase=None):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16*5*5)
        x = F.relu(self.fc1(x))
        encoded = F.relu(self.fc2(x))
        
        if phase=="train":
            decoded = self.decoder(encoded)
            decoded = decoded.view(decoded.size()[0], 3, 32, 32)
            return decoded
        
        x = self.fc3(encoded)
        return x
    
model = cnnDecoder()

In [4]:
import torch.optim as optim

criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [5]:
for epoch in range(10):
    
    running_loss = 0.0
    
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        inputs, labels = Variable(inputs), Variable(labels)
        ground_truth = inputs.clone()
        optimizer.zero_grad()
        outputs = model(inputs, phase="train")
        loss = criterion(outputs, ground_truth)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.data[0]
        
        if i % 2000 == 1999:
            print('[%d, %5d] loss: %.3f' % (epoch+1, i+1, running_loss/2000))
            running_loss = 0.0
            
print("Finished Training")

[1,  2000] loss: 0.139
[2,  2000] loss: 0.099
[3,  2000] loss: 0.091
[4,  2000] loss: 0.088
[5,  2000] loss: 0.083
[6,  2000] loss: 0.078
[7,  2000] loss: 0.076
[8,  2000] loss: 0.072
[9,  2000] loss: 0.071
[10,  2000] loss: 0.071
Finished Training


In [6]:
correct = 0
total = 0
for data in testloader:
    images, labels = data
    outputs = model(Variable(images))
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))

Accuracy of the network on the 10000 test images: 11 %
